**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
suppressMessages(suppressWarnings(library("edgeR")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_ENCODE_FCC/references 



In [7]:
txt_region = "fcc_astarr_macs_input_overlap"
txt_fdiry = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    "*",
    "overlap_count",
    "summary"
)
txt_fname = "data_list_count_column.rds"
txt_fglob = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
print(vec)

[1] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_ATAC_K562_Reddy_KS274/overlap_count/summary/data_list_count_column.rds"  
[2] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_ATAC_K562_Reddy_KS91/overlap_count/summary/data_list_count_column.rds"   
[3] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_ATAC_K562_Reddy_KSMerge/overlap_count/summary/data_list_count_column.rds"
[4] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_WHG_K562_Reddy_A001/overlap_count/summary/data_list_count_column.rds"    


## Test ASTARR MACS Overlap + ASTARR KS91

In [2]:
### init
txt_region = "fcc_astarr_macs_input_overlap"
txt_assay  = "STARR_ATAC_K562_Reddy_KS91"

### set directory
txt_fdiry = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    txt_assay,
    "overlap_count",
    "summary"
)
txt_fname = "data_list_count_column.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read object
lst_dat = readRDS(txt_fpath)
dat_cnt = lst_dat$data_cnt
dat_col = lst_dat$data_col

### arrange column data
dat_col = dat_col  %>% 
    dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
    column_to_rownames(var = "Sample")

### arrange count matrix
mat = dat_cnt %>%
    dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
    column_to_rownames(var = "Region")

dge = DGEList(counts=mat, group=dat_col$Group)
idx = filterByExpr(dge)
dge = dge[idx, , keep.lib.sizes=FALSE]
mat = dge$counts

mat_cnt = mat
print(nrow(dat_cnt))
print(nrow(mat_cnt))

[1] 150041
[1] 150040


In [3]:
### create DESeq2 object
dds = DESeqDataSetFromMatrix(
    countData = mat_cnt, 
    colData   = dat_col, 
    design    = ~Group)

### ensure Input group is set as reference
dds$Group <- relevel(dds$Group, ref = "Input")

### run deseq analysis
dds = DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [4]:
### save deseq object
txt_fdiry = dirname(txt_fpath)
txt_fname = "data_deseq2.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

obj = dds
saveRDS(obj, txt_fpath)

In [5]:
head(mat_cnt)

,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
chr1:10038-10405,145,207,244,229,178,206,5,12,4,10
chr1:14282-14614,106,139,153,137,123,128,8,23,26,41
chr1:16025-16338,124,189,175,170,178,164,8,7,15,18
chr1:17288-17689,259,350,400,367,369,331,7,13,23,59
chr1:28934-29499,258,324,349,324,346,310,8,11,3,26
chr1:115429-115969,950,1174,1276,1151,1179,1054,234,443,469,752


In [6]:
head(dat_col)

,FName,Assay,Prefix,Group
,<chr>,<chr>,<chr>,<fct>
Input.rep1,ASTARRseq_K562_KS91.hg38.Input.rep1.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep2,ASTARRseq_K562_KS91.hg38.Input.rep2.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep3,ASTARRseq_K562_KS91.hg38.Input.rep3.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep4,ASTARRseq_K562_KS91.hg38.Input.rep4.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep5,ASTARRseq_K562_KS91.hg38.Input.rep5.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input
Input.rep6,ASTARRseq_K562_KS91.hg38.Input.rep6.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS91,ASTARRseq_K562_KS91,Input


## Test ASTARR MACS Overlap + ASTARR KS274

In [8]:
### init
txt_region = "fcc_astarr_macs_input_overlap"
txt_assay  = "STARR_ATAC_K562_Reddy_KS274"

### set directory
txt_fdiry = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    txt_assay,
    "overlap_count",
    "summary"
)
txt_fname = "data_list_count_column.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read object
lst_dat = readRDS(txt_fpath)
dat_cnt = lst_dat$data_cnt
dat_col = lst_dat$data_col

### arrange column data
dat_col = dat_col  %>% 
    dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
    column_to_rownames(var = "Sample")

### arrange count matrix
mat = dat_cnt %>%
    dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
    column_to_rownames(var = "Region")

dge = DGEList(counts=mat, group=dat_col$Group)
idx = filterByExpr(dge)
dge = dge[idx, , keep.lib.sizes=FALSE]
mat = dge$counts

mat_cnt = mat
print(nrow(dat_cnt))
print(nrow(mat_cnt))

[1] 150042
[1] 150041


In [9]:
### create DESeq2 object
dds = DESeqDataSetFromMatrix(
    countData = mat_cnt, 
    colData   = dat_col, 
    design    = ~Group)

### ensure Input group is set as reference
dds$Group <- relevel(dds$Group, ref = "Input")

### run deseq analysis
dds = DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [10]:
txt_fdiry = dirname(txt_fpath)
print(txt_fdiry)

[1] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_ATAC_K562_Reddy_KS274/overlap_count/summary"


In [11]:
### save deseq object
txt_fdiry = dirname(txt_fpath)
txt_fname = "data_deseq2.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

obj = dds
saveRDS(obj, txt_fpath)

In [12]:
head(mat_cnt)

,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3
chr1:10038-10405,145,207,244,229,178,206,0,0,0
chr1:14282-14614,106,139,153,137,123,128,4,6,5
chr1:16025-16338,124,189,175,170,178,164,15,4,4
chr1:17288-17689,259,350,400,367,369,331,17,15,14
chr1:28934-29499,258,324,349,324,346,310,2,1,3
chr1:115429-115969,950,1174,1276,1151,1179,1054,290,265,277


In [13]:
head(dat_col)

,FName,Assay,Prefix,Group
,<chr>,<chr>,<chr>,<fct>
Input.rep1,ASTARRseq_K562_KS91.hg38.Input.rep1.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS274,ASTARRseq_K562_KS91,Input
Input.rep2,ASTARRseq_K562_KS91.hg38.Input.rep2.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS274,ASTARRseq_K562_KS91,Input
Input.rep3,ASTARRseq_K562_KS91.hg38.Input.rep3.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS274,ASTARRseq_K562_KS91,Input
Input.rep4,ASTARRseq_K562_KS91.hg38.Input.rep4.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS274,ASTARRseq_K562_KS91,Input
Input.rep5,ASTARRseq_K562_KS91.hg38.Input.rep5.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS274,ASTARRseq_K562_KS91,Input
Input.rep6,ASTARRseq_K562_KS91.hg38.Input.rep6.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KS274,ASTARRseq_K562_KS91,Input


## Test ASTARR MACS Overlap + ASTARR KSMerge

In [14]:
### init
txt_region = "fcc_astarr_macs_input_overlap"
txt_assay  = "STARR_ATAC_K562_Reddy_KSMerge"

### set directory
txt_fdiry = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    txt_assay,
    "overlap_count",
    "summary"
)
txt_fname = "data_list_count_column.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read object
lst_dat = readRDS(txt_fpath)
dat_cnt = lst_dat$data_cnt
dat_col = lst_dat$data_col

### arrange column data
dat_col = dat_col  %>% 
    dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
    column_to_rownames(var = "Sample")

### arrange count matrix
mat = dat_cnt %>%
    dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
    column_to_rownames(var = "Region")

dge = DGEList(counts=mat, group=dat_col$Group)
idx = filterByExpr(dge)
dge = dge[idx, , keep.lib.sizes=FALSE]
mat = dge$counts

mat_cnt = mat
print(nrow(dat_cnt))
print(nrow(mat_cnt))

[1] 150042
[1] 150040


In [17]:
head(mat_cnt)

,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4,Output.rep5,Output.rep6,Output.rep7
chr1:10038-10405,145,207,244,229,178,206,5,12,4,10,0,0,0
chr1:14282-14614,106,139,153,137,123,128,8,23,26,41,4,6,5
chr1:16025-16338,124,189,175,170,178,164,8,7,15,18,15,4,4
chr1:17288-17689,259,350,400,367,369,331,7,13,23,59,17,15,14
chr1:28934-29499,258,324,349,324,346,310,8,11,3,26,2,1,3
chr1:115429-115969,950,1174,1276,1151,1179,1054,234,443,469,752,290,265,277


In [19]:
dat_col

,FName,Assay,Prefix,Group
,<chr>,<chr>,<chr>,<fct>
Input.rep1,ASTARRseq_K562_KS91.hg38.Input.rep1.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Input
Input.rep2,ASTARRseq_K562_KS91.hg38.Input.rep2.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Input
Input.rep3,ASTARRseq_K562_KS91.hg38.Input.rep3.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Input
Input.rep4,ASTARRseq_K562_KS91.hg38.Input.rep4.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Input
Input.rep5,ASTARRseq_K562_KS91.hg38.Input.rep5.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Input
Input.rep6,ASTARRseq_K562_KS91.hg38.Input.rep6.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Input
Output.rep1,ASTARRseq_K562_KS91.hg38.Output.rep1.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Output
Output.rep2,ASTARRseq_K562_KS91.hg38.Output.rep2.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Output
Output.rep3,ASTARRseq_K562_KS91.hg38.Output.rep3.WGS.unstranded.bed.gz,STARR_ATAC_K562_Reddy_KSMerge,ASTARRseq_K562_KS91,Output


In [21]:
### create DESeq2 object
dds = DESeqDataSetFromMatrix(
    countData = mat_cnt, 
    colData   = dat_col, 
    design    = ~Group+Prefix)

### ensure Input group is set as reference
dds$Group <- relevel(dds$Group, ref = "Input")

### run deseq analysis
dds = DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [20]:
res = results(dds, contrast = c("Group", "Output", "Input"))
res

log2 fold change (MLE): Group Output vs Input 
Wald test p-value: Group Output vs Input 
DataFrame with 150040 rows and 6 columns
                          baseMean log2FoldChange     lfcSE      stat
                         <numeric>      <numeric> <numeric> <numeric>
chr1:10038-10405           37.1501      -2.841742  0.284172 -10.00008
chr1:14282-14614           38.1342      -0.422697  0.226805  -1.86370
chr1:16025-16338           39.0636      -1.364749  0.191184  -7.13841
chr1:17288-17689           78.4934      -1.381308  0.141734  -9.74578
chr1:28934-29499           59.3694      -2.709320  0.208606 -12.98772
...                            ...            ...       ...       ...
chrX:156000431-156003126 1086.0954       0.418280 0.0677411   6.17469
chrX:156009723-156010194   33.7977      -1.167201 0.2158986  -5.40625
chrX:156016432-156016780   29.9501      -4.597370 0.5152203  -8.92312
chrX:156024963-156025554  104.6954      -0.581278 0.1302340  -4.46334
chrX:156030352-156030709   31.

In [22]:
res = results(dds, contrast = c("Group", "Output", "Input"))
res

log2 fold change (MLE): Group Output vs Input 
Wald test p-value: Group Output vs Input 
DataFrame with 150040 rows and 6 columns
                          baseMean log2FoldChange     lfcSE       stat
                         <numeric>      <numeric> <numeric>  <numeric>
chr1:10038-10405           37.1501     -2.2374091  0.281460  -7.949302
chr1:14282-14614           38.1342      0.0272619  0.166605   0.163632
chr1:16025-16338           39.0636     -1.3419408  0.226576  -5.922701
chr1:17288-17689           78.4934     -1.3371407  0.165021  -8.102831
chr1:28934-29499           59.3694     -2.2857276  0.217494 -10.509365
...                            ...            ...       ...        ...
chrX:156000431-156003126 1086.0954       0.530026 0.0424995   12.47133
chrX:156009723-156010194   33.7977      -1.782659 0.2854546   -6.24498
chrX:156016432-156016780   29.9501      -4.203190 0.5501279   -7.64039
chrX:156024963-156025554  104.6954      -0.349008 0.1144266   -3.05006
chrX:156030352-156

In [23]:
txt_fdiry = dirname(txt_fpath)
print(txt_fdiry)

[1] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_ATAC_K562_Reddy_KSMerge/overlap_count/summary"


In [24]:
### save deseq object
txt_fdiry = dirname(txt_fpath)
txt_fname = "data_deseq2.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

obj = dds
saveRDS(obj, txt_fpath)

In [ ]:
### init
txt_region = "fcc_astarr_macs_input_overlap"
txt_assay  = "STARR_ATAC_K562_Reddy_KS91"

### set directory
txt_fdiry = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    txt_assay,
    "overlap_count",
    "summary"
)
txt_fname = "data_list_count_column.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read object
lst_dat = readRDS(txt_fpath)
dat_cnt = lst_dat$data_cnt
dat_col = lst_dat$data_col

### arrange column data
dat_col = dat_col  %>% 
    dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
    column_to_rownames(var = "Sample")

### arrange count matrix
mat = dat_cnt %>%
    dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
    column_to_rownames(var = "Region")

dge = DGEList(counts=mat, group=dat_col$Group)
idx = filterByExpr(dge)
dge = dge[idx, , keep.lib.sizes=FALSE]
mat = dge$counts

mat_cnt = mat
print(nrow(dat_cnt))
print(nrow(mat_cnt))

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [ ]:
print(dim(mat_cnt))
print(dim(mat))

In [14]:
txt_fdiry = dirname(txt_fpath)
txt_fdiry

[1] "/mount/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/STARR_ATAC_K562_Reddy_KS91/overlap_count/summary"

In [15]:
### save deseq object
txt_fdiry = dirname(txt_fpath)
txt_fname = "data_deseq2.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

obj = dds
saveRDS(obj, txt_fpath)

In [17]:
### create DESeq2 object
dds = DESeqDataSetFromMatrix(
    countData = mat_cnt, 
    colData   = dat_col, 
    design    = ~Group+Prefix)

### ensure Input group is set as reference
dds$Group <- relevel(dds$Group, ref = "Input")

### run deseq analysis
dds = DESeq(dds)

dds2 = dds

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


ERROR: Error in checkFullRank(modelMatrix): the model matrix is not full rank, so the model cannot be fit as specified.
  One or more variables or interaction terms in the design formula are linear
  combinations of the others and must be removed.

  Please read the vignette section 'Model matrix not full rank':

  vignette('DESeq2')


In [2]:
txt_fdiry = file.path(FD_RES, "assay_fcc")
dir(txt_fdiry)

[1] "MPRA_Tiling_K562_Tewhey_hannah" "STARR_ATAC_K562_Reddy_KS91"    
[3] "STARR_WHG_K562_Reddy_A001"

In [3]:
VEC_TXT_ASSAY = c(
    "STARR_ATAC_K562_Reddy_KS91",
    "STARR_WHG_K562_Reddy_A001"
)

In [4]:
for (txt_assay in VEC_TXT_ASSAY) {
    cat(txt_assay, "\n")
    txt_fdiry = file.path(FD_RES, "assay_fcc", txt_assay, "coverage", "*", "*", "summary")
    txt_fname = "data_list_count_column.rds"
    txt_fglob = file.path(txt_fdiry, txt_fname)
    
    vec_txt_fpath = Sys.glob(txt_fglob)
    
    for (txt_fpath in vec_txt_fpath){
        txt_fdiry = dirname(txt_fpath)
        print(txt_fdiry)
    }
    cat("\n")
}

STARR_ATAC_K562_Reddy_KS91 
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF333TAT/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF558BLC/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF925CYR/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF948AFM/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/dnase_ENCFF185XRG/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/dnase_ENCFF274YGF/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/cover